In [1]:
import os
import nd2
import napari
from napari.layers import Image, Labels, Shapes, Points
from napari_builtins.io import csv_to_layer_data

import numpy as np

from magicgui import magicgui

#from skimage.registration import phase_cross_correlation
from scipy.ndimage import affine_transform

In [2]:
def get_affine_from_points(x, x_prime):
    X = np.kron(np.eye(3), np.concat([x, np.ones((x.shape[0], 1))], axis=1)[:, np.newaxis, :]).reshape(x.shape[0]*3, 12)
    res = np.linalg.lstsq(X, x_prime.flatten())
    return get_affine(res[0]), res

def get_affine(a):
    affine = np.eye(4)
    affine[:3] = a.reshape(3, 4)
    return affine

In [3]:
viewers = [napari.Viewer(title=f"t{i}") for i in range(2)]
alignment_viewer = napari.Viewer(title="alignment")

In [4]:
root_path = r"Z:\Dasha\2. FAST-AB biosensors\Microscope\Inverted bladder model\NCCRBM_114\To analyze for the figure"
files = [f"Fig.4_h_TrimFABS_ANOX_mScarlet_insert{i}" for i in [1, 3, 4]]

In [5]:
@magicgui(
    file={"widget_type": "ComboBox", "choices": files},
    position={"widget_type": "ComboBox", "choices": range(3)},
)
def widget(file, position, viewer: napari.Viewer) -> list[napari.layers.Layer]:
    viewer.layers.clear()
    nd2_file = nd2.ND2File(os.path.join(root_path, file + ".nd2"))
    
    imgs = nd2_file.to_dask()[:, int(position)]

    gfps = imgs[:, :, 2].compute()
    mScarlets = imgs[:, :, 1].compute()

    viewers[1].add_image(
        gfps[1], name="gfp t1", colormap="cyan", blending="additive"
    )
    viewers[1].add_image(
        mScarlets[1], name="mScarlet t1", colormap="yellow", blending="additive"
    )

    try:
        datas = [
            csv_to_layer_data(os.path.join(root_path, "alignment", f"{file}_m{position}_t{i}.csv"))[0]
            for i in range(2)
        ]
    except FileNotFoundError:
        datas = [None, None]
    
    viewers[1].add_points(datas[1], name="t1", ndim=3)

    return [
        Image(gfps[0], name="gfp t0", colormap="cyan", blending="additive", metadata={"file": file, "position": position}),
        Image(mScarlets[0], name="mScarlet t0", colormap="yellow", blending="additive"),
        Points(datas[0], name="t0", ndim=3),
    ]

viewers[0].window.add_dock_widget(widget)

In [8]:
@viewers[0].bind_key('s', overwrite=True)
def save(viewer):
    metadata = viewers[0].layers[0].metadata
    position = metadata["position"]
    file = metadata["file"]

    for i in range(2):
        viewers[i].layers[2].save(os.path.join(
            root_path, "alignment", f"{file}_m{position}_t{i}.csv"
        ))

    datas = [v.layers[1].data for v in viewers]
    points = [v.layers[2].data for v in viewers]
    
    affine = get_affine_from_points(*points)[0]
    
    warped_affine = affine_transform(
        datas[0],
        np.linalg.inv(affine),
        output_shape=datas[0].shape,
        mode="constant", cval=np.nan,
        order=1,
    )

    alignment_viewer.layers.clear()
    alignment_viewer.add_image(warped_affine, name="warped t0", blending="additive", colormap="cyan")
    alignment_viewer.add_image(datas[1], name="t1", blending="additive", colormap="magenta")

HI
HI
